# Desafío - Clasificación desde la econometría

![Imagen%201.PNG](attachment:Imagen%201.PNG)

# Desafío 1: Preparar el ambiente de trabajo

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf


path_casita = "southafricanheart.csv"
path_colab = "/content/drive/MyDrive/Data Scientist/Desafío Latam/2 .-Fundamentos Data Science/6 .- Clasificación/Desafíos/southafricanheart.csv"
df = pd.read_csv(path_casita, index_col = 0)
df.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,160,12.00,5.73,23.11,Present,49,25.30,97.20,52,1
2,144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,Present,52,29.14,3.81,46,0
4,170,7.50,6.41,38.03,Present,51,31.99,24.26,58,1
5,134,13.60,3.50,27.78,Present,60,25.99,57.34,49,1


In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

Lista de las variables importadas

In [3]:
list(df.columns)

['sbp',
 'tobacco',
 'ldl',
 'adiposity',
 'famhist',
 'typea',
 'obesity',
 'alcohol',
 'age',
 'chd']

In [4]:
df.describe()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,53.103896,26.044113,17.044394,42.816017,0.346320
std,20.496317,4.593024,2.070909,7.780699,9.817534,4.213680,24.481059,14.608956,0.476313
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,124.000000,0.052500,3.282500,19.775000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,134.000000,2.000000,4.340000,26.115000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,148.000000,5.500000,5.790000,31.227500,60.000000,28.497500,23.892500,55.000000,1.000000
max,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000,1.000000


Distribución de categorías para las variables <code>famhist</code> y <code>chd</code>

In [5]:
print(df['famhist'].value_counts())

print(df['chd'].value_counts())

Absent     270
Present    192
Name: famhist, dtype: int64
0    302
1    160
Name: chd, dtype: int64


# Desafío 2

Recodifique <code>famhist</code> a dummy, asignando 1 a la categoría minoritaria.

In [6]:
df['famhist'].replace({'Present':1,'Absent':0}, inplace = True)
df.head()

,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
1,160,12.00,5.73,23.11,1,49,25.30,97.20,52,1
2,144,0.01,4.41,28.61,0,55,28.87,2.06,63,1
3,118,0.08,3.48,32.28,1,52,29.14,3.81,46,0
4,170,7.50,6.41,38.03,1,51,31.99,24.26,58,1
5,134,13.60,3.50,27.78,1,60,25.99,57.34,49,1


 Utilice <code>smf.logit</code> para estimar el modelo.

In [7]:
mod1 = smf.logit('chd ~ famhist', data = df).fit()
mod1.summary()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      460
Method:                           MLE   Df Model:                            1
Date:                Sat, 04 Sep 2021   Pseudo R-squ.:                 0.05740
Time:                        18:29:26   Log-Likelihood:                -280.95
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 4.937e-09
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.1690      0.143     -8.169      0.000      -1.449      -0.889
famhist        1.1690      0.203      5.751      0.000       0.771       1.567
==============================================================================
"""

Implemente una función <code>inverse_logit</code> que realice el mapeo de log-odds a probabilidad

In [8]:
def inverse_logit(x):
    return 1/(1+np.exp(-x))

Prediga lo siguiente:

In [9]:
def prediccion1(famhist):
    return mod1.predict(pd.DataFrame({'famhist':[famhist]}))

def prediccion2(famhist):
    return inverse_logit(mod1.params['Intercept']+mod1.params['famhist']*famhist)

a) ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tene runa enfermedad coronaria?

In [10]:
print(list(prediccion1(1) == prediccion2(1)))
list(prediccion1(1))

[True]


[0.4999999999999999]

b) ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?

In [11]:
print(list(prediccion1(0) == prediccion2(0)))
list(prediccion1(0))

[True]


[0.23703703703703705]

c) ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?

In [12]:
list(abs(prediccion1(0) - prediccion1(1)))

[0.26296296296296284]

d) Replique el modelo con <code>smf.ols</code>y comente las similitudes entre los coeficientes estimados.

In [13]:
mod2 = smf.ols('chd ~ famhist', data = df).fit()
mod2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    chd   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     36.86
Date:                Sat, 04 Sep 2021   Prob (F-statistic):           2.66e-09
Time:                        18:29:26   Log-Likelihood:                -294.59
No. Observations:                 462   AIC:                             593.2
Df Residuals:                     460   BIC:                             601.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2370      0.028      8.489      0.000       0.182       0.292
famhist        0.2630      0.043      6.071      0.000       0.178       0.348
==============================================================================
Omnibus:                      768.898   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               58.778
Skew:                           0.579   Prob(JB):                     1.72e-13
Kurtosis:                       1.692   Cond. No.                         2.47
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [14]:
mod2.params['famhist']/mod1.params['famhist']

0.22494826209023785

Aproximadamentte el beta de famhist del modelo de regresión lineal es apenas el 0.225 porciento del beta del beta de regresión logística de famhist

# Desafío 3: Estimación completa

a)

In [15]:
tmp = smf.logit('chd ~ sbp +tobacco + ldl + adiposity + famhist + typea + obesity + alcohol + age', data = df).fit()
tmp.summary()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            9
Date:                Sat, 04 Sep 2021   Pseudo R-squ.:                  0.2080
Time:                        18:29:26   Log-Likelihood:                -236.07
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 2.055e-22
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.1507      1.308     -4.701      0.000      -8.715      -3.587
sbp            0.0065      0.006      1.135      0.256      -0.005       0.018
tobacco        0.0794      0.027      2.984      0.003       0.027       0.132
ldl            0.1739      0.060      2.915      0.004       0.057       0.291
adiposity      0.0186      0.029      0.635      0.526      -0.039       0.076
famhist        0.9254      0.228      4.061      0.000       0.479       1.372
typea          0.0396      0.012      3.214      0.001       0.015       0.064
obesity       -0.0629      0.044     -1.422      0.155      -0.150       0.024
alcohol        0.0001      0.004      0.027      0.978      -0.009       0.009
age            0.0452      0.012      3.728      0.000       0.021       0.069
==============================================================================
"""

In [16]:
pvals = pd.DataFrame(tmp.pvalues)

var_determinantes = pvals[pvals[0] < 0.025]
list(var_determinantes.index)

['Intercept', 'tobacco', 'ldl', 'famhist', 'typea', 'age']

In [17]:
mod3 = smf.logit('chd ~ tobacco + ldl + famhist + typea + age', data = df).fit()
mod3.summary()

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      456
Method:                           MLE   Df Model:                            5
Date:                Sat, 04 Sep 2021   Pseudo R-squ.:                  0.2020
Time:                        18:29:26   Log-Likelihood:                -237.84
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 2.554e-24
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.4464      0.921     -7.000      0.000      -8.251      -4.642
tobacco        0.0804      0.026      3.106      0.002       0.030       0.131
ldl            0.1620      0.055      2.947      0.003       0.054       0.270
famhist        0.9082      0.226      4.023      0.000       0.466       1.351
typea          0.0371      0.012      3.051      0.002       0.013       0.061
age            0.0505      0.010      4.944      0.000       0.030       0.070
==============================================================================
"""

b) Compare los estadísticos de bondad de ajuste entre ambos.

In [18]:
tmp.prsquared-mod3.prsquared

0.005947820131952275

Comentario: El Pseudo R-Square es apenas un 0.5% mayor en el caso del primer modelo. (son bien malitos los 2)

c) Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria

In [19]:
df[['tobacco', 'ldl', 'famhist', 'typea', 'age','chd']].corr()['chd'].sort_values(ascending = False).drop('chd')

age        0.372973
tobacco    0.299718
famhist    0.272373
ldl        0.263053
typea      0.103156
Name: chd, dtype: float64

In [30]:
varss = ['tobacco', 'ldl', 'famhist', 'typea', 'age'] # Columnas del modelo mod3
vector = pd.DataFrame(mod3.params).T.drop(columns = 'Intercept') #DataFrame con Betas de mod3
r = pd.DataFrame(columns = ['tobacco', 'ldl', 'famhist', 'typea', 'age']) #DataFrame donde se guardarán las respuestas
for i in varss:
    vect_movil = pd.DataFrame(mod3.params).T.drop(columns = 'Intercept')
    for j in varss:
        if i != j:
            vect_movil[j] = 0
        else:
            vect_movil[j] = df[i].mean() # Reemplazo la diagonal con el promedio de la muestra
    r.loc[i] = list(vect_movil.loc[0])
r

,tobacco,ldl,famhist,typea,age
tobacco,3.635649,0.000000,0.000000,0.000000,0.000000
ldl,0.000000,4.740325,0.000000,0.000000,0.000000
famhist,0.000000,0.000000,0.415584,0.000000,0.000000
typea,0.000000,0.000000,0.000000,53.103896,0.000000
age,0.000000,0.000000,0.000000,0.000000,42.816017


In [31]:
lista = []

for i in r.index:
    lista.append(mod3.predict(pd.DataFrame(r.loc[i]).T)[0]) # Predigo según cada fila del dataframe del paso de arriba

r['efecto_sucinto'] = lista
r

,tobacco,ldl,famhist,typea,age,efecto_sucinto
tobacco,3.635649,0.000000,0.000000,0.000000,0.000000,0.002120
ldl,0.000000,4.740325,0.000000,0.000000,0.000000,0.003407
famhist,0.000000,0.000000,0.415584,0.000000,0.000000,0.002308
typea,0.000000,0.000000,0.000000,53.103896,0.000000,0.011257
age,0.000000,0.000000,0.000000,0.000000,42.816017,0.013574


In [32]:
r['efecto_sucinto']

tobacco    0.002120
ldl        0.003407
famhist    0.002308
typea      0.011257
age        0.013574
Name: efecto_sucinto, dtype: float64

In [33]:
r['efecto_sucinto'].sum()

0.03266551602059853

*Comentario extra*: Este sería la forma de decir qué variables están influyendo más en la variable dependiente 'chd'

# Desafío 4: Estimación de perfiles

a) La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.

In [22]:
cuatro_a = pd.DataFrame(df[['tobacco', 'ldl', 'famhist', 'typea', 'age']].mean()).T
cuatro_a

,tobacco,ldl,famhist,typea,age
0,3.635649,4.740325,0.415584,53.103896,42.816017


In [23]:
mod3.predict(cuatro_a)

0    0.293709
dtype: float64

In [24]:
inverse_logit(
    mod3.params['Intercept']+mod3.params['tobacco']*df['tobacco'].mean()+mod3.params['ldl']*df['ldl'].mean()+mod3.params['famhist']*df['famhist'].mean()+mod3.params['typea']*df['typea'].mean()+mod3.params['age']*df['age'].mean())

0.29370927481586945

b) La probabilidad de tener una enfermedad coronaria para un individuo con altos niveles de lipoproteína de baja densidad,​manteniendo todas las demás características constantes.

In [25]:
cuatro_b = cuatro_a.drop(columns = 'ldl')
cuatro_b['ldl'] = np.max(df['ldl'])
cuatro_b

,tobacco,famhist,typea,age,ldl
0,3.635649,0.415584,53.103896,42.816017,15.33


In [26]:
list(mod3.predict(cuatro_b))

[0.6980443104466211]

c) La probabilidad de tener una enfermedad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad,​manteniendo todas las demás características constantes.

In [27]:
cuatro_c = cuatro_a.drop(columns = 'ldl')
cuatro_c['ldl'] = np.min(df['ldl'])
cuatro_c

,tobacco,famhist,typea,age,ldl
0,3.635649,0.415584,53.103896,42.816017,0.98


In [28]:
list(mod3.predict(cuatro_c))

[0.18443595575404634]